In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [2]:
def show_frequency(frame,featrue):
    dfr=pd.DataFrame()
    Frequency=frame[featrue].value_counts() 
    total_count=len(frame)
    percentage=Frequency/total_count * 100
    percentage=percentage.round(2)
    dfr['Frequency']=frame[featrue].value_counts() 
    dfr['%']=percentage
    return dfr 

In [3]:
df_train=pd.read_csv('data.csv')
df_train=df_train.sample(frac=1)
df_test=pd.read_csv('testdata.csv')
submission=df_test
frames=[df_train,df_test]
#df=pd.concat(frames)


In [4]:
qmark='?'
df_train.replace(qmark,np.NaN,inplace=True)
df_test.replace(qmark,np.NaN,inplace=True)
#df_train=df_train.sample(frac=1)

In [5]:
def null_values(DataFrame_Name):
    
    sum_null = DataFrame_Name.isnull().sum()
    total_count = DataFrame_Name.isnull().count()
    percent_nullvalues = sum_null/total_count * 100
    df_null = pd.DataFrame()
    df_null['Total_values'] = total_count
    df_null['Null_Count'] = sum_null
    df_null['Percent'] = percent_nullvalues
    df_null = df_null.sort_values(by='Null_Count',ascending = False)

    return(df_null)

In [6]:
#--------------------------------Checking for NULL values ------------------------------------------------------

In [7]:
#null_values(df_train)

In [8]:
#null_values(df_test)

In [9]:
# ---------------------------- Checking column data types before filling null values ------------------------------

In [10]:
#df_train.dtypes

In [11]:
#df_test.dtypes

In [12]:


df_train['A2']=df_train['A2'].astype(str).astype(float)
df_test['A2']=df_test['A2'].astype(str).astype(float)


df_train['A14']=df_train['A14'].astype(str).astype(float)
df_test['A14']=df_test['A14'].astype(str).astype(float)

In [13]:
#-------------------------- filling Null values ----------------------------------------------------------

In [14]:
A2_mean=df_train['A2'].mean()
A14_mean=df_train['A14'].mean()

A1_max_occurrence=df_train['A1'].value_counts().index[0]
A6_max_occurrence=df_train['A6'].value_counts().index[0]
A9_max_occurrence=df_train['A9'].value_counts().index[0]
A3_max_occurrence=df_train['A3'].value_counts().index[0]
A4_max_occurrence=df_train['A4'].value_counts().index[0]

df_train=df_train.fillna({'A2':A2_mean,'A14':A14_mean,'A1':A1_max_occurrence,'A6':A6_max_occurrence,'A9':A9_max_occurrence,'A3':A3_max_occurrence,'A4':A4_max_occurrence})
df_test=df_test.fillna({'A1':A1_max_occurrence,'A6':A6_max_occurrence,'A9':A9_max_occurrence,'A14':A14_mean,'A2':A2_mean,'A3':A3_max_occurrence,'A4':A4_max_occurrence})


In [15]:
#------------------------- preprocess 1 --------------------------------------------------------------------------

# print(show_frequency(df_train,'A3'))
#    Frequency      %
# u         11  78.57
# y          3  21.43

# print(show_frequency(df_train,'A3'))
#    Frequency      %
# u        420  76.09
# y        130  23.55
# l          2   0.36

# since there are only 2 entries as l in 'A3' of the training set, replace them with U
df_train['A3'] = df_train['A3'].replace('l','u')



In [16]:
#------------------------- preprocess 2 --------------------------------------------------------------------------

# print(show_frequency(df_test,'A4'))
#    Frequency      %
# g         11  78.57
# p          3  21.43

# print(show_frequency(df_train,'A4'))
#     Frequency      %
# g         420  76.09
# p         130  23.55
# gg          2   0.36


# since there are only 2 entries as l in 'A4' of the training set, replace them with U
df_train['A4'] = df_train['A3'].replace('gg','g')

In [17]:
#------------------------- preprocess 3 --------------------------------------------------------------------------
# since A3,A4 are duplicated drop A4
df_train=df_train.drop(['A4'], axis=1)
df_test=df_test.drop(['A4'], axis=1)

In [18]:
# ----------------------- Encode caragorical data -----------------------------------------------------------

In [19]:
# binary encodings (Without True/False) can be done to ------> A1,A3

# encode A16 lables success =1 failure =0
df_train['A16'] = df_train['A16'].map({label:idx for idx,label in enumerate(np.unique(df_train['A16']))})

# encode A1 lables a =0 b =1 
df_train['A1'] = df_train['A1'].map({label:idx for idx,label in enumerate(np.unique(df_train['A1']))})
df_test['A1'] = df_test['A1'].map({label:idx for idx,label in enumerate(np.unique(df_test['A1']))})

# encode A8 lables false =0 true =1
df_train['A8'] = df_train['A8'].map({label:idx for idx,label in enumerate(np.unique(df_train['A8']))})
df_test['A8'] = df_test['A8'].map({label:idx for idx,label in enumerate(np.unique(df_test['A8']))})

# encode A11 lables false =0 true =1
df_train['A11'] = df_train['A11'].map({label:idx for idx,label in enumerate(np.unique(df_train['A11']))})
df_test['A11'] = df_test['A11'].map({label:idx for idx,label in enumerate(np.unique(df_test['A11']))})

# encode A13 lables false =0 true =1
df_train['A13'] = df_train['A13'].map({label:idx for idx,label in enumerate(np.unique(df_train['A13']))})
df_test['A13'] = df_test['A13'].map({label:idx for idx,label in enumerate(np.unique(df_test['A13']))})

In [20]:
df_test.head()

,A1,A2,A3,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,1,32.67,y,9.00,w,0,0,h,5.25,1,0,1,154.0,g
1,0,28.08,y,15.00,e,13212,0,z,0.00,1,0,0,0.0,g
2,1,73.42,u,17.75,ff,0,0,ff,0.00,1,0,1,0.0,g
3,1,64.08,u,20.00,x,1000,1,h,17.50,1,9,1,0.0,g
4,1,51.58,u,15.00,c,0,1,v,8.50,1,9,0,0.0,g


In [21]:
x_A3=df_train['A3'].map({label:idx for idx,label in enumerate(np.unique(df_train['A3']))})
#x_A4=df_train['A4'].map({label:idx for idx,label in enumerate(np.unique(df_train['A4']))})
x_A6=pd.get_dummies(df_train[['A6']])
x_A9=pd.get_dummies(df_train[['A9']])
x_A15=pd.get_dummies(df_train[['A15']])

df_train=pd.concat([df_train['A1'],df_train['A2'],x_A3,df_train['A5'],x_A6,df_train['A7'],df_train['A8'],x_A9,df_train['A10'],df_train['A11'],df_train['A12'],df_train['A13'],df_train['A14'],x_A15,df_train['A16']],axis=1)
df_train.head()

,A1,A2,A3,A5,A6_aa,A6_c,A6_cc,A6_d,A6_e,A6_ff,...,A9_z,A10,A11,A12,A13,A14,A15_g,A15_p,A15_s,A16
171,1,40.580000,0,5.00,0,1,0,0,0,0,...,0,5.000,1,7,0,0.000000,1,0,0,1
429,0,31.976661,0,11.25,0,0,0,0,0,1,...,0,0.000,0,0,0,186.928044,1,0,0,0
403,0,26.580000,1,2.54,0,0,0,0,0,1,...,0,0.000,0,0,1,180.000000,1,0,0,0
319,1,27.670000,0,0.75,0,0,0,0,0,0,...,0,0.165,0,0,1,220.000000,1,0,0,0
331,1,26.750000,1,4.50,0,1,0,0,0,0,...,0,2.500,0,0,0,200.000000,1,0,0,0


In [22]:
x_A3=df_test['A3'].map({label:idx for idx,label in enumerate(np.unique(df_test['A3']))})
#x_A4=df_test['A4'].map({label:idx for idx,label in enumerate(np.unique(df_test['A4']))})
x_A6=pd.get_dummies(df_test[['A6']])
x_A9=pd.get_dummies(df_test[['A9']])
x_A15=pd.get_dummies(df_test[['A15']])

df_test=pd.concat([df_test['A1'],df_test['A2'],x_A3,df_test['A5'],x_A6,df_test['A7'],df_test['A8'],x_A9,df_test['A10'],df_test['A11'],df_test['A12'],df_test['A13'],df_test['A14'],x_A15],axis=1)

df_test.head()

,A1,A2,A3,A5,A6_aa,A6_c,A6_cc,A6_d,A6_e,A6_ff,...,A9_v,A9_z,A10,A11,A12,A13,A14,A15_g,A15_p,A15_s
0,1,32.67,1,9.00,0,0,0,0,0,0,...,0,0,5.25,1,0,1,154.0,1,0,0
1,0,28.08,1,15.00,0,0,0,0,1,0,...,0,1,0.00,1,0,0,0.0,1,0,0
2,1,73.42,0,17.75,0,0,0,0,0,1,...,0,0,0.00,1,0,1,0.0,1,0,0
3,1,64.08,0,20.00,0,0,0,0,0,0,...,0,0,17.50,1,9,1,0.0,1,0,0
4,1,51.58,0,15.00,0,1,0,0,0,0,...,1,0,8.50,1,9,0,0.0,1,0,0


In [23]:
for col in df_train.columns: 
    print(col,end=' ,') 
print('-------------------')
for col in df_test.columns: 
    print(col,end=' ,') 

A1 ,A2 ,A3 ,A5 ,A6_aa ,A6_c ,A6_cc ,A6_d ,A6_e ,A6_ff ,A6_i ,A6_j ,A6_k ,A6_m ,A6_q ,A6_r ,A6_w ,A6_x ,A7 ,A8 ,A9_bb ,A9_dd ,A9_ff ,A9_h ,A9_j ,A9_n ,A9_o ,A9_v ,A9_z ,A10 ,A11 ,A12 ,A13 ,A14 ,A15_g ,A15_p ,A15_s ,A16 ,-------------------
A1 ,A2 ,A3 ,A5 ,A6_aa ,A6_c ,A6_cc ,A6_d ,A6_e ,A6_ff ,A6_i ,A6_j ,A6_k ,A6_m ,A6_q ,A6_r ,A6_w ,A6_x ,A7 ,A8 ,A9_bb ,A9_ff ,A9_h ,A9_j ,A9_n ,A9_v ,A9_z ,A10 ,A11 ,A12 ,A13 ,A14 ,A15_g ,A15_p ,A15_s ,

In [24]:
def fill_missing_columns(df_train,df_test):
    col_train=list(df_train)
    col_test=list(df_test)
    size=len(col_train)
   
    for i in range(size):
        col=col_train[i]
        if col not in col_test:
            df_test.insert(i,col,0)
    
            
fill_missing_columns(df_train,df_test)

In [25]:
# for col in df_train.columns: 
#     print(col,end=' ,') 
# print('-------------------')
# for col in df_test.columns: 
#     print(col,end=' ,') 

In [26]:
df_test=df_test.drop(['A16'], axis=1)
 

In [27]:
feature_cols=df_train.columns
feature_cols=feature_cols[0:len(feature_cols)-1]

#split dataset in features and target variable
X = df_train[feature_cols] # Features
y = df_train.A16 # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# X_train = df_train[feature_cols] # Features
# y_train = df_train.A16 # Target variable

In [28]:
#----- Feature scaling-------------------
#-------------- Feature scaling --------------------------------------------
# random forest, decision trees do not need feature scaling

stdsc = StandardScaler()
X_train = stdsc.fit_transform(X_train)
X_test = stdsc.transform(X_test)
df_test=stdsc.transform(df_test)


In [29]:
#-------------------apply different modles -----------------------

In [30]:
# Create Decision Tree classifer object
dt = DecisionTreeClassifier()
dt = dt.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = dt.predict(X_test)
dt_accuracy=metrics.accuracy_score(y_test, y_pred)
dt_tree_precision=metrics.precision_score(y_test, y_pred)
dt_tree_recall=metrics.recall_score(y_test, y_pred)
print('Accuracy= ',metrics.accuracy_score(y_test, y_pred))
print('Precision= ',metrics.precision_score(y_test, y_pred))
print('Recall= ',metrics.recall_score(y_test, y_pred))

Accuracy=  0.8433734939759037
Precision=  0.7701149425287356
Recall=  0.9178082191780822


In [31]:
#Create a Gaussian Classifier
gnb = GaussianNB()
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)
gnb_accuracy=metrics.accuracy_score(y_test, y_pred)
gnb_precision=metrics.precision_score(y_test, y_pred)
gnb_recall=metrics.recall_score(y_test, y_pred)
print('Accuracy= ',metrics.accuracy_score(y_test, y_pred))
print('Precision= ',metrics.precision_score(y_test, y_pred))
print('Recall= ',metrics.recall_score(y_test, y_pred))

Accuracy=  0.6987951807228916
Precision=  0.896551724137931
Recall=  0.3561643835616438


In [32]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)
knn_accuracy=metrics.accuracy_score(y_test, y_pred)
knn_precision=metrics.precision_score(y_test, y_pred)
knn_recall=metrics.recall_score(y_test, y_pred)
print('Accuracy= ',metrics.accuracy_score(y_test, y_pred))
print('Precision= ',metrics.precision_score(y_test, y_pred))
print('Recall= ',metrics.recall_score(y_test, y_pred))

Accuracy=  0.7891566265060241
Precision=  0.7435897435897436
Recall=  0.7945205479452054


In [33]:
svm = svm.SVC(kernel='linear') # Linear Kernel
svm.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = svm.predict(X_test)
svm_accuracy=metrics.accuracy_score(y_test, y_pred)
svm_precision=metrics.precision_score(y_test, y_pred)
svm_recall=metrics.recall_score(y_test, y_pred)
print('Accuracy= ',metrics.accuracy_score(y_test, y_pred))
print('Precision= ',metrics.precision_score(y_test, y_pred))
print('Recall= ',metrics.recall_score(y_test, y_pred))

Accuracy=  0.8253012048192772
Precision=  0.7558139534883721
Recall=  0.8904109589041096


In [34]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = lr.predict(X_test)
lr_accuracy=metrics.accuracy_score(y_test, y_pred)
lr_precision=metrics.precision_score(y_test, y_pred)
lr_recall=metrics.recall_score(y_test, y_pred)
print('Accuracy= ',metrics.accuracy_score(y_test, y_pred))
print('Precision= ',metrics.precision_score(y_test, y_pred))
print('Recall= ',metrics.recall_score(y_test, y_pred))

Accuracy=  0.8253012048192772
Precision=  0.7619047619047619
Recall=  0.8767123287671232


In [35]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = rf.predict(X_test)
rf_accuracy=metrics.accuracy_score(y_test, y_pred)
rf_precision=metrics.precision_score(y_test, y_pred)
rf_recall=metrics.recall_score(y_test, y_pred)
print('Accuracy= ',metrics.accuracy_score(y_test, y_pred))
print('Precision= ',metrics.precision_score(y_test, y_pred))
print('Recall= ',metrics.recall_score(y_test, y_pred))

Accuracy=  0.8674698795180723
Precision=  0.8311688311688312
Recall=  0.8767123287671232


In [36]:
#--------------------------------------------reuslts------------------------------------------

In [37]:
op_rf=rf.predict(df_test)
print(op_rf)

[1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1]


In [38]:
op_lr=lr.predict(df_test)
print(op_lr)


[1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 0 1 0 0 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1]


In [39]:
op_svm=svm.predict(df_test)
print(op_svm)


[1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1]


In [40]:
op_dt=dt.predict(df_test)
print(op_dt)


[1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]


In [41]:
submission['A16']=op_rf
submission.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,32.67,y,p,9.00,w,0,False,h,5.25,True,0,True,154.0,g,1
1,a,28.08,y,p,15.00,e,13212,False,z,0.00,True,0,False,0.0,g,1
2,b,73.42,u,g,17.75,ff,0,False,ff,0.00,True,0,True,0.0,g,0
3,b,64.08,u,g,20.00,x,1000,True,h,17.50,True,9,True,0.0,g,1
4,b,51.58,u,g,15.00,c,0,True,v,8.50,True,9,False,0.0,g,1


In [44]:
# df_submission=pd.Dataframe();
# df_submission['Category']
# submission.to_csv("Group_7.csv")

def submit(op):
    df_submission=pd.DataFrame(columns=['Id','Category'])
    ins=len(op)
    ids=[int(i) for i in range(1,ins+1)]
    df_submission['Id']=ids
    cat=[]
    for i in op:
        if i==0:
            cat.append('Failure')
        elif i==1:
            cat.append('Success')
    df_submission['Category']=cat
#     for i in range(1,ins+1):
#         df_submission['Id'][i-1]=i
    
    df_submission.to_csv("Group_7.csv",index=False)
    return df_submission.tail()

In [45]:
submit(op_rf)

,Id,Category
133,134,Success
134,135,Success
135,136,Success
136,137,Failure
137,138,Success
